# Preparing Time Series Features and Labels

In [1]:
#import paskages 
import tensorflow as tf

In [2]:
#generatort a tf dataset
dataset = tf.data.Dataset.range(10)

for val in dataset:
    print(val.numpy())

0
1
2
3
4
5
6
7
8
9


In [3]:
# Generate a tf dataset with 10 elements (i.e. numbers 0 to 9)
dataset = tf.data.Dataset.range(10)

dataset= dataset.window(size=5 , shift =1)

#window the data
for window_dataset in dataset:
    print(window_dataset)

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [4]:
for window_dataset in dataset:
    print([item.numpy() for item in window_dataset])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
[6, 7, 8, 9]
[7, 8, 9]
[8, 9]
[9]


In [6]:
#ex2
dataset_2=tf.data.Dataset.range(30)

dataset_2 = dataset_2.window(shift=1 , size=5)

for window_dataset in dataset_2:
    print([item.numpy() for item in window_dataset])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
[6, 7, 8, 9, 10]
[7, 8, 9, 10, 11]
[8, 9, 10, 11, 12]
[9, 10, 11, 12, 13]
[10, 11, 12, 13, 14]
[11, 12, 13, 14, 15]
[12, 13, 14, 15, 16]
[13, 14, 15, 16, 17]
[14, 15, 16, 17, 18]
[15, 16, 17, 18, 19]
[16, 17, 18, 19, 20]
[17, 18, 19, 20, 21]
[18, 19, 20, 21, 22]
[19, 20, 21, 22, 23]
[20, 21, 22, 23, 24]
[21, 22, 23, 24, 25]
[22, 23, 24, 25, 26]
[23, 24, 25, 26, 27]
[24, 25, 26, 27, 28]
[25, 26, 27, 28, 29]
[26, 27, 28, 29]
[27, 28, 29]
[28, 29]
[29]


 You can use the drop_remainder flag to make sure that only 5-element windows are retained.

In [9]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(size=5 ,shift= 1, drop_remainder=True)

for window_dataset in dataset:
    print([item.numpy() for item in window_dataset])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


##### Flatten the Windows

In training the model later, you will want to prepare the windows to be tensors instead of the Dataset structure. You can do that by feeding a mapping function to the flat_map() method. This function will be applied to each window and the results will be flattened into a single dataset. To illustrate, the code below will put all elements of a window into a single batch then flatten the result.

In [25]:
dataset= tf.data.Dataset.range(10)

dataset= dataset.window(5 ,shift=1 , drop_remainder=True)

dataset=dataset.flat_map(lambda window: window.batch(5))

for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


###### Group into features and labels

Next, you will want to mark the labels in each window. For this exercise, you will do that by splitting the last element of each window from the first four. This is done with the map() method containing a lambda function that defines the window slicing.

In [22]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(5))

dataset = dataset.map(lambda window: (window[:-1], window[-1]))

for x,y in dataset:
    print("x = ",x.numpy())
    print("y = ",y.numpy())
    print()

x =  [0 1 2 3]
y =  4

x =  [1 2 3 4]
y =  5

x =  [2 3 4 5]
y =  6

x =  [3 4 5 6]
y =  7

x =  [4 5 6 7]
y =  8

x =  [5 6 7 8]
y =  9



###### Shuffle the data

It is good practice to shuffle your dataset to reduce sequence bias while training your model. This refers to the neural network overfitting to the order of inputs and consequently, it will not perform well when it does not see that particular order when testing. You don't want the sequence of training inputs to impact the network this way so it's good to shuffle them up.

You can simply use the shuffle() method to do this. The buffer_size parameter is required for that and as mentioned in the doc, you should put a number equal or greater than the total number of elements for better shuffling. We can see from the previous cells that the total number of windows in the dataset is 6 so we can choose this number or higher.

In [27]:
dataset= tf.data.Dataset.range(10)

dataset=dataset.window(5 ,shift=1 ,drop_remainder=True)

dataset= dataset.flat_map(lambda window: window.batch(5))

dataset = dataset.map(lambda window: (window[:-1], window[-1]))

dataset= dataset.shuffle(buffer_size=10)

for x,y in dataset:
    print("x = ",x.numpy())
    print('y = ',y.numpy())
    print()

x =  [0 1 2 3]
y =  4

x =  [5 6 7 8]
y =  9

x =  [4 5 6 7]
y =  8

x =  [2 3 4 5]
y =  6

x =  [1 2 3 4]
y =  5

x =  [3 4 5 6]
y =  7



###### Create batches for training

In [39]:
dataset = tf.data.Dataset.range(10)

# Window the data but only take those with the specified size
dataset = dataset.window(5, shift=1, drop_remainder=True)

# Flatten the windows by putting its elements in a single batch
dataset = dataset.flat_map(lambda window: window.batch(5))

# Create tuples with features (first four elements of the window) and labels (last element)
dataset = dataset.map(lambda window: (window[:-1], window[-1]))

# Shuffle the windows
dataset = dataset.shuffle(buffer_size=10)

# Create batches of windows
dataset = dataset.batch(2).prefetch(1)

# Print the results
for x,y in dataset:
    print("x = ", x.numpy())
    print("y = ", y.numpy())
    print()

x =  [[0 1 2 3]
 [4 5 6 7]]
y =  [4 8]

x =  [[3 4 5 6]
 [5 6 7 8]]
y =  [7 9]

x =  [[1 2 3 4]
 [2 3 4 5]]
y =  [5 6]

